In [1]:

import numpy as np
import pandas as pd
from gensim.models.keyedvectors import KeyedVectors
from nltk.corpus import stopwords
from sklearn.ensemble import RandomForestClassifier
import sklearn.metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from sklearn.metrics import classification_report
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
import re

import joblib

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/lucaspancotto/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/lucaspancotto/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/lucaspancotto/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
def sui_dep(x):
    if x=='depression':
        return 1
    return 0

reddit  = pd.read_csv('../../../data/reddit_depression_suicidewatch_clean.csv').drop(columns=['Unnamed: 0'])
reddit['label'] = reddit.label.apply(sui_dep)
reddit

,text,label
0,breakup friend talk thing hurt lose feeling pa...,1
1,navigate feeling feeling stretch understand pl...,1
2,bf month told depressed week happened feel bot...,1
3,exhausted finally rest thing starting settle h...,0
4,severly bullied till depressed misanthrope mul...,1
...,...,...
20358,seroquel hour drank panicking ik sound joke se...,0
20359,happened week week depression insomnia sleep g...,1
20360,feel house people room loneliness suck,1
20361,politician neighbor corporation society cultur...,1


In [3]:
reddit.dropna(inplace=True)
reddit.isnull().sum()

text     0
label    0
dtype: int64

In [4]:
y = reddit.label.values
y

array([1, 1, 1, ..., 1, 1, 1])

In [5]:
X = reddit.text
X = X.astype(str)
X

0        breakup friend talk thing hurt lose feeling pa...
1        navigate feeling feeling stretch understand pl...
2        bf month told depressed week happened feel bot...
3        exhausted finally rest thing starting settle h...
4        severly bullied till depressed misanthrope mul...
                               ...                        
20358    seroquel hour drank panicking ik sound joke se...
20359    happened week week depression insomnia sleep g...
20360               feel house people room loneliness suck
20361    politician neighbor corporation society cultur...
20362    feel existing depressed girlfriend year broken...
Name: text, Length: 20340, dtype: object

In [6]:
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=500, split=' ') 
tokenizer.fit_on_texts(X)

X = tokenizer.texts_to_sequences(X)

X = pad_sequences(X, value=-1000, maxlen = 1348 , dtype='float64')

vocab_size = len(tokenizer.word_index) + 1
print("Total words", vocab_size)

2022-03-17 13:30:57.788981: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-17 13:30:57.789026: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Total words 30428


In [7]:
X = np.reshape(X, (X.shape[0], 1, X.shape[1]))
X.shape

#X.reshape(( 1, 1348))

(20340, 1, 1348)

In [8]:
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.optimizers import Adam
from keras.layers import Activation, Dense
from keras.layers import Input

model = joblib.load('../../../transcripciones_finalizado_model.joblib')

'''def init_model_2():
  model = Sequential()
  #input = Input(shape=(142, 1353))
  initial_learning_rate = 0.001
  lr_schedule = ExponentialDecay(initial_learning_rate, decay_steps=2000, decay_rate=0.5)
  adam = Adam(learning_rate=lr_schedule)
  model.add(layers.Masking(mask_value=-1000))
  #model.add(layers.Embedding(500, 7059, input_length = X.shape[1]))
  model.add(layers.GRU(512, return_sequences=True, activation='tanh')) #input_shape = [1, 1353]))
  model.add(layers.GRU(256, return_sequences=True, activation='tanh'))
  model.add(layers.GRU(128, return_sequences=True, activation='tanh'))
  model.add(layers.GRU(256, return_sequences=True, activation='tanh'))
  model.add(layers.GRU(128, return_sequences=True, activation='tanh'))
  model.add(layers.GRU(64, return_sequences=True, activation='tanh'))
  model.add(layers.GRU(32, return_sequences=True, activation='tanh',dropout=0.01))
  model.add(layers.Flatten())
  model.add(Dense(16, activation = 'tanh'))
  model.add(Dense(1, activation = 'sigmoid'))
  
  model.compile(
      optimizer=adam,
      loss="binary_crossentropy",
      metrics=['accuracy', 'AUC','Precision','Recall'])
  return model

model = init_model_2()


from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=10, restore_best_weights=True)
history_fit = model.fit(X, y, 
          batch_size = 32,
          epochs=100,
          validation_data = (X_val, Y_val),
          #callbacks=[es],
          verbose = 1
         )'''

2022-03-17 13:31:01.104228: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-17 13:31:01.104277: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-17 13:31:01.104297: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-QMHRL2V): /proc/driver/nvidia/version does not exist
2022-03-17 13:31:01.104533: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-17 13:31:04.829573: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'cond/while' ha

2022-03-17 13:31:14.057491: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'cond/while' has 13 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2022-03-17 13:31:14.074507: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'cond' has 4 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2022-03-17 13:31:15.751034: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'cond/while' has 13 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2022-03-17 13:31:15.768318: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'cond' has 4 outputs but the _output_shapes attribute specifies shapes for 46 outputs. Output shapes may be inaccurate.
2022-03-17 13:31:15.790632: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'cond/while' has 13 outputs but the _output_sh

'def init_model_2():\n  model = Sequential()\n  #input = Input(shape=(142, 1353))\n  initial_learning_rate = 0.001\n  lr_schedule = ExponentialDecay(initial_learning_rate, decay_steps=2000, decay_rate=0.5)\n  adam = Adam(learning_rate=lr_schedule)\n  model.add(layers.Masking(mask_value=-1000))\n  #model.add(layers.Embedding(500, 7059, input_length = X.shape[1]))\n  model.add(layers.GRU(512, return_sequences=True, activation=\'tanh\')) #input_shape = [1, 1353]))\n  model.add(layers.GRU(256, return_sequences=True, activation=\'tanh\'))\n  model.add(layers.GRU(128, return_sequences=True, activation=\'tanh\'))\n  model.add(layers.GRU(256, return_sequences=True, activation=\'tanh\'))\n  model.add(layers.GRU(128, return_sequences=True, activation=\'tanh\'))\n  model.add(layers.GRU(64, return_sequences=True, activation=\'tanh\'))\n  model.add(layers.GRU(32, return_sequences=True, activation=\'tanh\',dropout=0.01))\n  model.add(layers.Flatten())\n  model.add(Dense(16, activation = \'tanh\'))\n

In [9]:
len(y)==len(X)
model

In [10]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=10, restore_best_weights=True)

history_fit = model.fit(x=X,
    y=y,
    batch_size=32,
    epochs=100,
    verbose=1,
    callbacks=es,
    validation_split=0.2,
    
)

Epoch 1/100
509/509 [==============================] - 57s 66ms/step - loss: 0.7021 - accuracy: 0.5103 - auc: 0.5078 - precision: 0.5146 - recall: 0.7451 - val_loss: 0.6945 - val_accuracy: 0.5029 - val_auc: 0.5003 - val_precision: 0.5028 - val_recall: 0.9956
Epoch 2/100
509/509 [==============================] - 24s 48ms/step - loss: 0.6931 - accuracy: 0.5053 - auc: 0.5028 - precision: 0.5109 - recall: 0.7677 - val_loss: 0.6964 - val_accuracy: 0.5027 - val_auc: 0.5003 - val_precision: 0.5027 - val_recall: 1.0000
Epoch 3/100
509/509 [==============================] - 23s 45ms/step - loss: 0.6932 - accuracy: 0.5119 - auc: 0.4924 - precision: 0.5117 - recall: 0.9934 - val_loss: 0.6932 - val_accuracy: 0.5029 - val_auc: 0.5003 - val_precision: 0.5028 - val_recall: 0.9956
Epoch 4/100
509/509 [==============================] - 25s 48ms/step - loss: 0.6930 - accuracy: 0.5110 - auc: 0.4967 - precision: 0.5113 - recall: 0.9897 - val_loss: 0.6933 - val_accuracy: 0.5029 - val_auc: 0.5003 - val_pre